### Learn about structured output

Here we will ask Agent to respond in a structured output. 

Read more on it at https://openai.github.io/openai-agents-python/agents/

Output Type section

In [ ]:
# Import libraries
from dotenv import load_dotenv
from agents import Runner, trace, function_tool, Agent, OpenAIChatCompletionsModel, SQLiteSession
from openai import AsyncOpenAI


# import BaseModel from Pydantic
from pydantic import BaseModel

#### Load openai api key from environment

In [ ]:
# Load environment variables
load_dotenv(override=True)

#### Define LLM model

In [ ]:
# Define LLM model

# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using the local LLM server client
# use gpt-oss which is better for tool usage and reasoning even though it takes time
model = OpenAIChatCompletionsModel(model = "gpt-oss",openai_client= client)


#### Define Tool

In [ ]:
from agents import function_tool

@function_tool
def calculate_n_content(dna_sequence: str) -> float:
    """Use this tool to calculate total N bases in a DNA sequence. 
    This is a custom tool which adds 10 to the N count.
    """
    n_count = dna_sequence.upper().count('N')
    return n_count+10

#### Define output type

In [ ]:
class Ncontent(BaseModel):
    total_N_bases: int
    dna_sequence: str

#### Define Agent

In [ ]:
agent_instructions = """You are a helpful assistant for Bioinformatics task. 
You are equipped with calculate_n_content tool. 
If query involves DNA sequence N content analysis, you must use this tool to calculate N content. Otherwise chat normally.
"""

bioinfo_agent = Agent(name="BioinformaticsAgent",
                    instructions=agent_instructions,
                    model="gpt-4.1-mini",
                    tools=[calculate_n_content], 
                    output_type=Ncontent)


#### Run Agent

In [ ]:
query="Calculate the N content in the following DNA sequence: ATGCNNNATGC"
result = await Runner.run(bioinfo_agent, query)
result.final_output

Ncontent(total_N_bases=13, dna_sequence='ATGCNNNATGC')